In [1]:
%cd /home/q123/Desktop/explo

import torch 
import gpytorch 
import logging
import logging.config

from src.helpers import setup_experiment
from src.trainer import Trainer
from src.optimizers.gibo import GIBOptimizer
from src.optimizers.vanilla import BOptimizer
from src.config import get_configs

logging.config.fileConfig('logging.conf')
# create root logger
logger = logging.getLogger()

/home/q123/Desktop/explo


/home/q123/miniconda3/envs/boptim/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
env_name = "Swimmer-v4"
kernel_name = "rbfstate" ## "linearstate" /"rbfstate"

env_config,likelihood_config,kernel_config,optimizer_config,trainer_config = get_configs(env_name,kernel_name)
additional_layers=[] ### can be empty or [8,7] for adding 2 layers with width 8,7 respectively


# likelihood_config = {
#                         "noise_hyperprior":gpytorch.priors.torch_priors.UniformPrior(a=0.05,b=0.5),
#                         "noise_constraint":gpytorch.constraints.constraints.Interval(0.05,0.5)
#                         }


# optimizer_config = {
#         "n_eval":1,
#         ### for GIBO
#         "n_max":20, 
#         "n_info_samples":8,
#         "delta":0.001, ## 0.01 better for linear
#         ### hessian normalisation applies only for rbf
#         "normalize_gradient":True if kernel_name == "rbf" else False,
#         "standard_deviation_scaling":False,
# }

kernel_config = {
        "use_ard":False,
        "kernel_name":kernel_name,
        # "lengthscale_constraint":gpytorch.constraints.constraints.Interval(0.01,10),
        # "lengthscale_hyperprior":gpytorch.priors.torch_priors.UniformPrior(0.01,10),
        # "outputscale_constraint":gpytorch.constraints.constraints.Interval(0.001,1),
        # "outputscale_hyperprior":gpytorch.priors.torch_priors.UniformPrior(0.001,1),
        }




trainer_config = {
        "n_steps" :50, 
        "report_freq":5,
        "save_best":False,
}


model,objective_env = setup_experiment(env_config,kernel_config,likelihood_config,additional_layers)


### Chose optimizer 
#optimizer = BOptimizer(**optimizer_config)
optimizer = GIBOptimizer(model,**optimizer_config)
trainer = Trainer(model,objective_env,optimizer,**trainer_config)
rslt= trainer.run()

MathLog.src.helpers : WARNING : MLP dimensions : [8, 2]


/home/q123/miniconda3/envs/boptim/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:69: UserWarning: WARN: Agent's minimum action space value is -infinity. This is probably too low.
  logger.warn(
/home/q123/miniconda3/envs/boptim/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:73: UserWarning: WARN: Agent's maximum action space value is infinity. This is probably too high
  logger.warn(
/home/q123/miniconda3/envs/boptim/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(


Using ard_num_dims = 18
 Gibo will use 100 last points to fit GP and 16 info samples


/home/q123/miniconda3/envs/boptim/lib/python3.10/site-packages/gpytorch/lazy/lazy_tensor.py:1741: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:1672.)
  Linv = torch.triangular_solve(Eye, L, upper=False).solution


current 0.42357781529426575 / max 0.9338468313217163 /batch_mean 0.6523505449295044 /batch_max 0.9338468313217163 
##############################
covar_lengthscale max 44.74371337890625 / min 44.74371337890625                      covar_outputscale 0.6931471824645996                     noise 0.014999999664723873
##############################
last parameters tensor([ 0.0558,  0.0131, -0.4108,  0.0292,  0.0892, -0.0668,  0.1500,  0.0954,
        -0.1035,  0.3700, -0.3680, -0.0535,  0.1998, -0.2350,  0.3152,  0.1840,
         0.1187, -0.0133])
MLL : 0.9189781546592712
current 0.9282147288322449 / max 0.976556658744812 /batch_mean 0.8658860325813293 /batch_max 0.9282147288322449 
##############################
covar_lengthscale max 28.297151565551758 / min 28.297151565551758                      covar_outputscale 0.06628325581550598                     noise 0.010002071969211102
##############################
last parameters tensor([ 0.3412, -0.0615, -0.6207, -0.0529,  0.3636, -0.3446,  